In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape
base_grande = base_grande.dropna()
base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')


liste_var = base_grande.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)
indicatrice_patho_niv1 = liste_bis

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)
classe_age = []
for i in base_grande['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
base_grande.loc[:,'classe_age'] = classe_age

In [ ]:
base_grande

In [ ]:
model_foie_alpha = base_grande[['dept', 'classe_age','indicatrice__Maladies du foie ou du pancréas (hors mucoviscidose)','sexe','region']]

In [ ]:
model_foie_alpha.columns = model_foie_alpha.columns.str.replace(' ', '_')

In [ ]:
model_foie_alpha

In [ ]:
resultats = model_foie_alpha[model_foie_alpha['indicatrice__Maladies_du_foie_ou_du_pancréas_(hors_mucoviscidose)'] == 1].groupby('classe_age').size()
resultats
#exactement le même problème que pour la respiration même nombre 
#fixed

In [ ]:
# Filtrez les lignes où la colonne 'indicatrice__Maladies_respiratoires_chroniques_(hors_mucoviscidose)' est égale à 1
T_filtered = model_foie_alpha[model_foie_alpha['indicatrice__Maladies_du_foie_ou_du_pancréas_(hors_mucoviscidose)'] == 1]

# Agrégation par âge pour compter le nombre d'individus
agg_data = T_filtered.groupby('classe_age').size().reset_index(name='nombre_de_cas')

# Créez un histogramme
plt.bar(agg_data['classe_age'], height=agg_data['nombre_de_cas'])

# Ajoutez des étiquettes et un titre
plt.xlabel('Classe d\'âge')
plt.ylabel('Nombre de cas de maladies du foie ou du pancréas chroniques')
plt.title('Nombre de cas de maladies du fois ou du pancréas chroniques par classe d\'âge')

# Affichez l'histogramme
plt.show()

In [ ]:
counts = model_foie_alpha['classe_age'].value_counts()
print(counts)